# 効果測定（回帰：digits.csv）
## １．CSVファイルの読み込みと表示。

In [100]:
# pandasをインポートし、別名 pd とする
import pandas as pd
# train_test_splitメソッドのインポート
from sklearn.model_selection import train_test_split

# CSV読み込み
DATAFILE_PATH = 'datafiles'
FILE_NAME = 'digits.csv'
df = pd.read_csv(f'{DATAFILE_PATH}/{FILE_NAME}', index_col=0)
df

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0,0,5,13,9,1,0,0,0,0,...,0,0,0,6,13,10,0,0,0,0
1,0,0,0,12,13,5,0,0,0,0,...,0,0,0,0,11,16,10,0,0,1
2,0,0,0,4,15,12,0,0,0,0,...,0,0,0,0,3,11,16,9,0,2
3,0,0,7,15,13,1,0,0,0,8,...,0,0,0,7,13,13,9,0,0,3
4,0,0,0,1,11,0,0,0,0,0,...,0,0,0,0,2,16,4,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792,0,0,4,10,13,6,0,0,0,1,...,0,0,0,2,14,15,9,0,0,9
1793,0,0,6,16,13,11,1,0,0,0,...,0,0,0,6,16,14,6,0,0,0
1794,0,0,1,11,15,1,0,0,0,0,...,0,0,0,2,9,13,6,0,0,8
1795,0,0,2,10,7,0,0,0,0,0,...,0,0,0,5,12,16,12,0,0,9


## ４．前処理
### ■欠損値処理

In [101]:
#欠損値数 ->なし
df.isnull().sum()

pixel_0_0    0
pixel_0_1    0
pixel_0_2    0
pixel_0_3    0
pixel_0_4    0
            ..
pixel_7_4    0
pixel_7_5    0
pixel_7_6    0
pixel_7_7    0
target       0
Length: 65, dtype: int64

### ■特徴量と正解の分割

In [102]:
# 特徴量と正解データに分割
x = df.loc[:, :'pixel_7_7']
t = df['target']


### ■標準化

In [103]:
# 特徴量の標準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
new = sc.fit_transform(x)

### ■訓練データと検証データに分割

In [104]:
# 訓練データと検証用データに分割
x_train, x_val, y_train, y_val = train_test_split(new, t,
    test_size = 0.2, random_state = 0)

## ５．学習モデルの選択

### ■ロジスティック回帰

In [105]:
from sklearn.linear_model import LogisticRegression
# ロジスティック回帰による学習
lr_model = LogisticRegression(C = 0.1,random_state=0,
    multi_class = 'auto', solver = 'lbfgs')

In [106]:
# 正解率を確認
lr_model.fit(x_train, y_train)
print( lr_model.score(x_train, y_train) )
lr_model.score(x_val, y_val)

0.988865692414753


0.9638888888888889

### ■ランダムフォレスト

In [107]:
# ランダムフォレストのインポート
from sklearn.ensemble import RandomForestClassifier
x_train, x_test, y_train, y_test=train_test_split(x, t,
 test_size = 0.2, random_state = 0)
rfc_model = RandomForestClassifier(n_estimators = 200,
random_state = 0)

In [108]:
# 正解率を確認
rfc_model.fit(x_train, y_train)

print(rfc_model.score(x_train, y_train))
print(rfc_model.score(x_test, y_test))

1.0
0.9722222222222222


### ■アダブースト

In [109]:
# アダブーストのインポート
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier

x_train, x_test, y_train, y_test = train_test_split(x, t,
test_size = 0.2, random_state = 0)
# 最大の深さ10の決定木を何個も作っていく（10が一番正解率が高くなったため）
base_model = DecisionTreeClassifier(random_state = 0,
max_depth = 10)

# 決定木を230個作成（230が一番正解率が高くなったため）
abc_model = AdaBoostClassifier(n_estimators = 230,
random_state = 0, base_estimator = base_model)
abc_model.fit(x_train,y_train) # 学習

print(abc_model.score(x_train, y_train)) # 訓練データの正解率
print(abc_model.score(x_test, y_test)) # テストデータの正解率


1.0
0.9888888888888889


## ６．選択された学習モデルで学習

In [110]:
# アダブーストで実施
x_train, x_test, y_train, y_test = train_test_split(x, t,
test_size = 0.2, random_state = 0)
# 最大の深さ10の決定木を何個も作っていく（10が一番正解率が高くなったため）
base_model = DecisionTreeClassifier(random_state = 0,
max_depth = 10)

# アダブーストで決定木を230個作成（230が一番正解率が高くなったため）
abc_model = AdaBoostClassifier(n_estimators = 230,
random_state = 0, base_estimator = base_model)
abc_model.fit(x_train,y_train) # 学習

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10,
                                                         random_state=0),
                   n_estimators=230, random_state=0)

## ７．正解率または決定係数の表示

In [113]:
print(abc_model.score(x_train, y_train)) # 訓練データの正解率
print(abc_model.score(x_test, y_test)) # テストデータの正解率

1.0
0.9888888888888889


## ８．学習後のモデルを保存

In [112]:
import pickle
with open('digits_lr.pki','wb') as f:
  pickle.dump(lr_model,f)
with open('digits_rfc.pki','wb') as f:
  pickle.dump(rfc_model,f)
with open('digits_abc.pki','wb') as f:
  pickle.dump(abc_model,f)